## Import libraries

In [1]:
import jpype
import jpype.imports
DEVEL = '/usr/workspace/hangal1/RDA/RadSim/proj-radsim/jars/'
jpype.startJVM(classpath=["%s/*"%DEVEL])

In [2]:
from gov.llnl.ensdf.decay import BNLDecayLibrary
from gov.llnl.rtk.physics import EmissionCalculator, DecayCalculator, SourceImpl, Nuclides, Elements, ActivityUnit, EmissionConverter
from gov.llnl.rtk.physics import SphericalModel, LayerImpl, MaterialImpl
from gov.llnl.rtk.physics import Section, SphericalSection, CylindricalSection, ConicalSection
from gov.llnl.rtk.mcnp import MCNP_Photon, MCNP_Electron, RadSim_MCNP_Job
from gov.llnl.math.euclidean import Vector3
from gov.llnl.rtk.flux import FluxTrapezoid, FluxGroupTrapezoid

import numpy as np
import matplotlib.pyplot as plt
import re
from java.nio.file import Path
from java.util import ArrayList

from source_flux import parse_beta_file
from source_flux import get_flux_from_lines
from source_flux import add_fluxes
from geometry import get_offset_vector
from build_mcnp_job import build_mcnp_job

## Define source nuclide and get flux

In [3]:
do_betas = True

# Declare source list
source_nuclide = 'Cs137'
Bq = ActivityUnit.Bq
activity = 1e5 #Bq

sourceList = ArrayList()
sourceList.add(SourceImpl.fromActivity(Nuclides.get(source_nuclide), activity, Bq))

# Load the decay library
decay_lib = BNLDecayLibrary()
decay_lib.loadFile(Path.of("/usr/workspace/hangal1/RDA/RadSim/proj-radsim/src/gov.llnl.rtk.mcnp/data/BNL2023.txt"))

# Setup decay calculator
decayCalculator = DecayCalculator()
decayCalculator.setDecayLibrary(decay_lib)

# Age the source to secular equilibrium
eqbm_sourceList = decayCalculator.age(sourceList,10000) #?

# Setup emission calculator
emissionCalculator = EmissionCalculator()
emissionCalculator.setDecayLibrary(decay_lib)

# Calculate the emissions
emissions = emissionCalculator.calculate(eqbm_sourceList)

# Create the gamma emmision list
emissionList = ArrayList()
for gamma in emissions.getGammas():
    emissionList.add(gamma)    

#convert emission list to flux
line_flux = get_flux_from_lines(emissionList)
if(do_betas): beta_flux = parse_beta_file(source_nuclide, activity)

# get number of source particles
num_source_gammas = int(np.sum([gamma.getCounts() for gamma in line_flux.getPhotonGroups()]))
num_source_betas = int(np.sum([beta.getCounts() for beta in beta_flux.getPhotonGroups()]))

## Define source and environment geometry

### Define materials

In [4]:
def get_material(elements, atoms, density, name):
    #calculate mass
    total_mass = 0
    for element, atom in zip(elements,atoms):
        symbol = re.sub('[^a-zA-Z]+', '', element)
        total_mass += atom*(Elements.get(symbol).getMolarMass())
    
    #create Material
    material = MaterialImpl()
    for element, atom in zip(elements,atoms):   
        symbol = re.sub('[^a-zA-Z]+', '', element)
        element_mass = Elements.get(symbol).getMolarMass()
        material.addElement(element, atom*element_mass/total_mass, 0.0)
    
    material.setDensity(density)
    return material

# Define materials
#get_material(elements, atoms, density (g/cm3), name)
lead = get_material(['Pb208'], [1], 11.34, 'lead')
cs_oxide = get_material(['Cs137','O16'], [1,2], 4.65, 'cs_oxide')

### Define geometries

#### Supported geometries
#### Spherical
- SphericalSection(origin, axis, theta1, theta2, phi1, phi2, inner radius, outer radius)
- SphericalSection.Sphere(origin, radius)
- SphericalSection.HollowSphere(origin, innerRadius, outerRadius)
- SphericalSection.HalfHollowSphere(origin, axis, innerRadius, outerRadius)
- SphericalSection.HalfSphere(origin, axis, radius)
#### Cylindrical
- CylindricalSection(origin, axis, phi1, phi2, inner radius, outer radius, height)
- CylindricalSection.Cylinder(origin, axis, radius, height)
#### Conical
- ConicalSection(origin, axis, theta1, theta2, phi1, phi2, start length, end length)    
- ConicalSection.Cone(origin, axis, theta, length)

length in cm\
0 < theta < pi\
0 < phi < 2pi


In [5]:
# Define sections
sections = []    

#First section always source
#SphericalSection.Sphere(origin, radius)
origin = Vector3.ZERO
radius = 1.0
sphere = SphericalSection.Sphere(origin, radius)
sphere.setMaterial(cs_oxide)
sections.append(sphere)

#Shielding Sections from here on
#SphericalSection(origin, axis, theta1, theta2, phi1, phi2, inner radius, outer radius)
thickness = 0.01
origin = Vector3.ZERO
axis=Vector3.AXIS_Z
theta1, theta2 = 0.0, np.pi/2 #0 < theta < pi
phi1, phi2 = 0.0, 2*np.pi #0 < phi < 2pi
innerRadius = 1.5
outerRadius = 1.5 + thickness
cap = SphericalSection(origin, axis, theta1, theta2, phi1, phi2, innerRadius, outerRadius)
cap.setMaterial(lead)
sections.append(cap)

#CylindricalSection(origin, axis, phi1, phi2, inner radius, outer radius, height)
axis = Vector3.AXIS_Z
thickness = 0.01
origin = get_offset_vector(-3.0, axis)
phi1, phi2 = 0.0, np.pi/2 #0 < theta < 2pi
innerRadius = 1.5
outerRadius = 1.5 + thickness
height = 3.0
container = CylindricalSection(origin, axis, phi1, phi2, innerRadius, outerRadius, height)
container.setMaterial(lead)
sections.append(container)

#CylindricalSection.Cylinder(origin, axis, radius, height)
origin = get_offset_vector(-2.75, axis)
axis = Vector3.AXIS_Z
radius = 1.5
height = 0.5
floor = CylindricalSection.Cylinder(origin, axis, radius, height)
floor.setMaterial(lead)
sections.append(floor)

#ConicalSection(origin, axis, theta1, theta2, phi1, phi2, start length, end length)
#0 < theta < pi and 0 < phi < 2pi, axis points towards tapering section    
origin = Vector3.ZERO
axis = Vector3.AXIS_Z
reverse_axis = Vector3.of(-axis.getX(), -axis.getY(), -axis.getZ())
theta1, theta2 = np.pi/16, 5*np.pi/32 #0 < theta < pi
phi1, phi2 = 0.0, np.pi/2 #0 < theta < 2pi
startLength = 1.0
endLength = 2.25
holder = ConicalSection(origin, reverse_axis, theta1, theta2, phi1, phi2, startLength, endLength)
holder.setMaterial(lead)
sections.append(holder)

### Declare source geometry and shielding sections

In [6]:
# Get the source sections
source_section = sections[0]
shielding_sections = sections[1::]

## Define MCNP job

In [7]:
num_tasks=int(8)

# Set some particle options
p = MCNP_Photon()
e = MCNP_Electron()
e.setNumBremPhotonsPerStep(1000)
e.setNumBremPerStep(1000)

mcnp_path = '/collab/usr/gapps/mcnp/toss_4_x86_64_ib/bin/mcnp6'

results = {}

## Run MCNP job

In [ ]:
#declare beta flux
results['Beta Source'] = FluxTrapezoid()
results['Gamma From Betas'] = FluxTrapezoid()

lower_bins = []
upper_bins = []
for group in line_flux.getPhotonGroups():
    lower_bins.append(group.getEnergyLower())
    upper_bins.append(group.getEnergyUpper())
for lower, upper in zip(lower_bins, upper_bins):
    results['Beta Source'].addPhotonGroup(FluxGroupTrapezoid(lower, upper, 0, 0.0))
    results['Gamma From Betas'].addPhotonGroup(FluxGroupTrapezoid(lower, upper, 0, 0.0))

#addSurfaceCurrentTally(key, origin, radius) of sphere for tallying    
#process betas
if(do_betas):
    # MCNP Beta Run (No Detector Tally, No Source Geometry)
    job = build_mcnp_job(mcnp_path, beta_flux, e, [p], num_particles=num_source_betas)
    #addSurfaceCurrentTally(String key, Vector3 origin, double radius)
    job.addSurfaceCurrentTally("Source Tally", Vector3.of(0.0, 0.0, 0.0), 5.0)
    job.run(num_tasks)
    results['Beta Source'] = job.getTallySpectrum("Source Tally", e, True)

    # MCNP Beta Run (No Detector Tally)
    job = build_mcnp_job(mcnp_path, beta_flux, e, [p], source_section=None, sections=shielding_sections, num_particles=num_source_betas)
    job.addSurfaceCurrentTally("Source Tally", Vector3.of(0.0, 0.0, 0.0), 5.0)
    job.run(num_tasks)
    results['Gamma From Betas'] = job.getTallySpectrum("Source Tally", p, True)

# MCNP Gamma Run (No Detector Tally, No Source Geometry)
job = build_mcnp_job(mcnp_path, line_flux, p, num_particles=num_source_gammas)
job.addSurfaceCurrentTally("Source Tally", Vector3.of(0.0, 0.0, 0.0), 5.0)
job.run(num_tasks)
results['Gamma Source'] = job.getTallySpectrum("Source Tally", p, True)

# MCNP Gamma Run (No Detector Tally)
job = build_mcnp_job(mcnp_path, line_flux, p, source_section=None, sections=shielding_sections, num_particles=num_source_gammas)
job.addSurfaceCurrentTally("Source Tally", Vector3.of(0.0, 0.0, 0.0), 5.0)
job.run(num_tasks)
results['Attenuated Gammas'] = job.getTallySpectrum("Source Tally", p, True)

# MCNP detector transport (No Source Geometry)
total_flux = add_fluxes([results['Gamma From Betas'], results['Attenuated Gammas']])
job = build_mcnp_job(mcnp_path, total_flux, p, num_particles=num_source_gammas)
job.addSurfaceCurrentTally("Detector Tally", Vector3.of(0.0, 0.0, 10.0), 1.0)
job.run(num_tasks)
results['Gammas At Detector'] = job.getTallySpectrum("Detector Tally", p, False)           # Note the false indicates particles entering the sphere

/collab/usr/gapps/mcnp/toss_4_x86_64_ib/bin/mcnp6 i=/g/g90/hangal1/radsim_package/test_dir/RadSim_SourceGen_Test_1726865318532.input o=/g/g90/hangal1/radsim_package/test_dir/RadSim_SourceGen_Test_1726865318532.output run=/g/g90/hangal1/radsim_package/test_dir/RadSim_SourceGen_Test_1726865318532.run tasks 8
          Code Name & Version = mcnp6, 6.3.0, production
          Copyright Triad National Security, LLC/LANL/DOE - see LICENSE file
  
     _/      _/        _/_/_/       _/      _/       _/_/_/         _/_/_/ 
    _/_/  _/_/      _/             _/_/    _/       _/    _/     _/        
   _/  _/  _/      _/             _/  _/  _/       _/_/_/       _/_/_/     
  _/      _/      _/             _/    _/_/       _/           _/    _/    
 _/      _/        _/_/_/       _/      _/       _/             _/_/       
  
 mcnp6 ver=6.3.0, ld=05/12/23  09/20/24 13:48:38                   
Source version = release/6.3-6bfd83016b
  comment.  Physics models disabled.
 1 s 0.0 0.0 0.0 5.0       

### Plotting macro for RadSim flux

In [ ]:
def plot_spectra(flux, **kwargs):
    groups = flux.getPhotonGroups()
    bins = [groups[0].getEnergyLower()]
    counts = []
    for group in groups:
        bins.append(group.getEnergyUpper())
        counts.append(group.getCounts())
    bins = np.array(bins)
    centers = 0.5 * (bins[0:-1] + bins[1::])
    counts_per_keV = counts / (bins[1::] - bins[0:-1]) / 1e3
    plt.plot(centers, counts_per_keV, **kwargs)
    plt.ylabel('Counts per keV per source particle')
    plt.xlabel('Energy [keV]')

In [ ]:
# beta spectrum
if(do_betas):
    plt.clf()
    plot_spectra(results['Beta Source'], color='k')
    plt.title(source_nuclide+'Beta Spectrum')
    plt.ylim([10**-10, 10**-5])
    plt.xlim([0.0, 1500])
    plt.yscale('log')
    plt.show()

In [ ]:
# gammas from source 
plt.clf()
plot_spectra(results['Gamma Source'], color='k')
plt.title(source_nuclide+'Gamma Spectrum')
# plt.ylim([10**-10, 10**-5])
plt.xlim([0.0, 1500])
plt.yscale('log')
plt.show()

In [ ]:
#gammas with source geometry and detector tally
plt.clf()
plot_spectra(results['Gamma From Betas'], label='Gammas from Betas', color='r')
plot_spectra(results['Attenuated Gammas'], label='Direct Gammas', color='g')
plot_spectra(total_flux, label='Total', color='k')
plot_spectra(results['Gammas At Detector'], label='Gammas at Detector', color='b', ls='--')
plt.title(source_nuclide)
plt.ylim([10**-14, 10**-2])
plt.xlim([0.0, 1500])
plt.yscale('log')
plt.legend()
plt.show()